In [152]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from mord import LogisticIT
from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
from imblearn import under_sampling
from imblearn.under_sampling import NearMiss
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [75]:
CF = pd.read_csv("/Users/bradleybeck/Documents/MDA 620/creditcard.csv")

In [76]:
CF.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [77]:
CF.shape

(284807, 31)

In [78]:
CF.describe().round()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,...,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0,284807.0
mean,94814.0,0.0,0.0,-0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,88.0,0.0
std,47488.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,250.0,0.0
min,0.0,-56.0,-73.0,-48.0,-6.0,-114.0,-26.0,-44.0,-73.0,-13.0,...,-35.0,-11.0,-45.0,-3.0,-10.0,-3.0,-23.0,-15.0,0.0,0.0
25%,54202.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.0,-1.0,...,-0.0,-1.0,-0.0,-0.0,-0.0,-0.0,-0.0,-0.0,6.0,0.0
50%,84692.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,0.0,0.0,-0.0,...,-0.0,0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,22.0,0.0
75%,139320.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,77.0,0.0
max,172792.0,2.0,22.0,9.0,17.0,35.0,73.0,121.0,20.0,16.0,...,27.0,11.0,23.0,5.0,8.0,4.0,32.0,34.0,25691.0,1.0


In [79]:
CF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

# Removing NA's

In [80]:
CF.isna().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [81]:
CF.dtypes.value_counts()

float64    30
int64       1
dtype: int64

In [82]:
CF_categorical=CF.loc[:,CF.dtypes== object]
CF_categorical

""
0
1
2
3
4
...
284802
284803
284804
284805


In [83]:
CF_numerical = CF.select_dtypes(exclude=[object])
CF_numerical

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0


# Removing Duplicates

In [84]:
CF.duplicated(keep='first').sum() #there are 1081 duplicates

1081

In [85]:
CF[CF.duplicated()].head() #you can see below 113 and 114 are the same

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
33,26.0,-0.529912,0.873892,1.347247,0.145457,0.414209,0.100223,0.711206,0.176066,-0.286717,...,0.046949,0.208105,-0.185548,0.001031,0.098816,-0.552904,-0.073288,0.023307,6.14,0
35,26.0,-0.535388,0.865268,1.351076,0.147575,0.433680,0.086983,0.693039,0.179742,-0.285642,...,0.049526,0.206537,-0.187108,0.000753,0.098117,-0.553471,-0.078306,0.025427,1.77,0
113,74.0,1.038370,0.127486,0.184456,1.109950,0.441699,0.945283,-0.036715,0.350995,0.118950,...,0.102520,0.605089,0.023092,-0.626463,0.479120,-0.166937,0.081247,0.001192,1.18,0
114,74.0,1.038370,0.127486,0.184456,1.109950,0.441699,0.945283,-0.036715,0.350995,0.118950,...,0.102520,0.605089,0.023092,-0.626463,0.479120,-0.166937,0.081247,0.001192,1.18,0
115,74.0,1.038370,0.127486,0.184456,1.109950,0.441699,0.945283,-0.036715,0.350995,0.118950,...,0.102520,0.605089,0.023092,-0.626463,0.479120,-0.166937,0.081247,0.001192,1.18,0


In [86]:
CF_d=CF.copy()
CF_d.drop_duplicates(subset=None, inplace=True)

In [87]:
CF_d.shape

(283726, 31)

In [88]:
CF = CF_d
del CF_d

# Fraud Cases Detected

In [162]:
CF.Class.value_counts() #0 is no fraud and 1 is fraud

0    284315
1       492
Name: Class, dtype: int64

# Splitting the Data

In [90]:
x=CF.drop(['Class'],axis=1)
x.head()
x.shape

(283726, 30)

In [91]:
y=CF['Class']
y.head()
y.shape

(283726,)

# Scalling the Data

In [92]:
std_scaler = StandardScaler()  #the data must be scaled in order for the train test split to work 
rob_scaler = RobustScaler()

CF['scaled_amount'] = rob_scaler.fit_transform(CF['Amount'].values.reshape(-1,1))
CF['scaled_time'] = rob_scaler.fit_transform(CF['Time'].values.reshape(-1,1))

In [93]:
scaled_amount = CF['scaled_amount']
scaled_time = CF['scaled_time']

CF.drop(['scaled_amount', 'scaled_time'], axis=1, inplace=True)
CF.insert(0, 'scaled_amount', scaled_amount)
CF.insert(1, 'scaled_time', scaled_time)


In [94]:
CF.head()

,scaled_amount,scaled_time,Time,V1,V2,V3,V4,V5,V6,V7,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,1.774718,-0.995290,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,-0.268530,-0.995290,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,4.959811,-0.995279,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.411487,-0.995279,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,0.667362,-0.995267,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [96]:
CF = pd.read_csv("/Users/bradleybeck/Documents/MDA 620/creditcard.csv")
CF['scaledAmt'] = StandardScaler().fit_transform(np.array(CF['Amount']).reshape(-1,1))
CF['scaledTime'] = StandardScaler().fit_transform(np.array(CF['Time']).reshape(-1,1))
CF.insert(0, 'scaled_amount', CF['scaledAmt'])
CF.insert(1, 'scaled_time', CF['scaledTime'])
CF.drop(['Time', 'Amount', 'scaledAmt', 'scaledTime'], axis=1, inplace=True)

# Linear Regression

In [167]:
Y = CF['Class']
X = CF.drop(['Class'], axis=1, inplace=False)

In [168]:
np.random.seed(123)  #this randomizes the data 
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.70, test_size = 0.30, random_state=1)


print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(199364, 29)
(85443, 29)
(199364,)
(85443,)


In [169]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [170]:
print('Intercept:', regressor.intercept_)
print('slope:', regressor.coef_)

Intercept: 0.001725642694303232
slope: [-2.03367111e-03  2.72871376e-03 -5.10409434e-03  3.78693788e-03
 -2.49906298e-03 -1.59215970e-03 -6.76287350e-03  8.62425565e-04
 -3.67422479e-03 -8.10883970e-03  6.26072128e-03 -1.06425987e-02
 -2.26624569e-04 -1.30927341e-02 -1.20471158e-04 -9.26100811e-03
 -1.58901388e-02 -5.55650634e-03  1.88169837e-03  5.43700256e-04
  2.35023231e-03  8.82143454e-05  7.05070996e-05 -6.00125394e-04
  4.27943178e-04  4.37422896e-04  1.69908330e-03  1.17467946e-03
  1.32022483e-03]


In [171]:
y_pred = regressor.predict(X_test)
y_pred

array([ 0.00025076, -0.00155748,  0.00248441, ...,  0.00258006,
       -0.00055823, -0.00019625])

In [173]:
comparison_df = pd.DataFrame({"Actual":y_test,"Predicted":y_pred})
comparison_df.head()

,Actual,Predicted
169876,0,0.000251
127467,0,-0.001557
137900,0,0.002484
21513,0,0.000044
134700,0,0.007742


In [174]:
residuals = y_test - y_pred
residuals

169876   -0.000251
127467    0.001557
137900   -0.002484
21513    -0.000044
134700   -0.007742
            ...   
128956   -0.006451
177494    0.002648
26287    -0.002580
198160    0.000558
25893     0.000196
Name: Class, Length: 85443, dtype: float64

In [175]:
print('MAE:', mean_absolute_error(y_test,y_pred))
print("MSE",mean_squared_error(y_test,y_pred))
print("RMSE",np.sqrt(mean_squared_error(y_test,y_pred)))

MAE: 0.0033080484340579424
MSE 0.0008195920816987147
RMSE 0.028628518678037024


In [178]:
r2 = r2_score(y_test,y_pred)
print("R2:", r2)  

R2: 0.4804501758271654


In [136]:
import missingno as msno
msno.heatmap(CF)

/Users/bradleybeck/opt/anaconda3/lib/python3.9/site-packages/seaborn/matrix.py:305: UserWarning: Attempting to set identical left == right == 0 results in singular transformations; automatically expanding.
  ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))
/Users/bradleybeck/opt/anaconda3/lib/python3.9/site-packages/seaborn/matrix.py:305: UserWarning: Attempting to set identical bottom == top == 0 results in singular transformations; automatically expanding.
  ax.set(xlim=(0, self.data.shape[1]), ylim=(0, self.data.shape[0]))


<AxesSubplot:>

# Decision Tree


In [123]:
CF = pd.read_csv("/Users/bradleybeck/Documents/MDA 620/creditcard.csv")

In [132]:
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import pydotplus
from sklearn import tree
from dmba import plotDecisionTree, classificationSummary, regressionSummary

In [163]:
np.random.seed(123)
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.30, test_size = 0.70, random_state=1)

fullClassTree = DecisionTreeClassifier(random_state=1)
fullClassTree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=1)

In [164]:
classificationSummary(y_train, fullClassTree.predict(X_train))
classificationSummary(y_test, fullClassTree.predict(X_test))

Confusion Matrix (Accuracy 1.0000)

       Prediction
Actual     0     1
     0 85301     0
     1     0   141
Confusion Matrix (Accuracy 0.9991)

       Prediction
Actual      0      1
     0 198922     92
     1     94    257


In [135]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(fullClassTree.fit(X_train, y_train), filled=True)
plt.show()

/var/folders/bv/1pv1q79s2kx5vwvzgdpfkth00000gn/T/ipykernel_6886/1769691321.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


# Logistic Regression

In [137]:
CF = pd.read_csv("/Users/bradleybeck/Documents/MDA 620/creditcard.csv")

In [138]:
CF = CF.drop(['Time'], axis=1)

In [140]:
amt = np.array(CF['Amount'])
amt = amt.reshape(-1,1)
amt.shape
amt[:10]

array([[149.62],
       [  2.69],
       [378.66],
       [123.5 ],
       [ 69.99],
       [  3.67],
       [  4.99],
       [ 40.8 ],
       [ 93.2 ],
       [  3.68]])

In [141]:
sc = StandardScaler()
CF['Amount'] = sc.fit_transform(amt)

In [142]:
CF.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,0.244964,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-0.342475,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,1.160686,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,0.140534,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,-0.073403,0


In [146]:
x = CF.drop('Class', axis=1)
y = CF['Class']
x.shape, y.shape

((284807, 29), (284807,))

In [150]:
lr = LogisticRegression()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((199364, 29), (85443, 29), (199364,), (85443,))

In [154]:
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(x_train, y_train)

LogisticRegression(C=1e+42, solver='liblinear')

In [160]:
logit_reg_pred = logit_reg.predict(x_test)
logit_reg_proba = logit_reg.predict_proba(x_test)
logit_result = pd.DataFrame({'actual': y_test, 
                             'p(0)': [p[0] for p in logit_reg_proba],
                             'p(1)': [p[1] for p in logit_reg_proba],
                             'predicted': logit_reg_pred })
print(logit_result)

        actual      p(0)      p(1)  predicted
31773        0  0.999798  0.000202          0
117430       0  0.999479  0.000521          0
274383       0  0.999949  0.000051          0
216704       0  0.999496  0.000504          0
245854       0  0.999645  0.000355          0
...        ...       ...       ...        ...
77982        0  0.995268  0.004732          0
148012       0  0.999854  0.000146          0
241018       0  0.996193  0.003807          0
112976       0  0.999853  0.000147          0
171570       0  0.999444  0.000556          0

[85443 rows x 4 columns]


In [153]:
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[85284    11]
 [   46   102]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     85295
           1       0.90      0.69      0.78       148

    accuracy                           1.00     85443
   macro avg       0.95      0.84      0.89     85443
weighted avg       1.00      1.00      1.00     85443

0.9993328885923949
